In [1]:
import os
import sys
sys.path.append("../")
import tensorflow as tf
from glob import glob
from typing import List, Tuple
#from .data_processing.data_loader import dataset_loader
#from rosbag2numpy.data_processing.data_loader_costmap  import dataset_loader
from rosbag2numpy.data_processing.data_loader_dual_costmap  import dataset_loader

from matplotlib import pyplot as plt
import numpy as np
from numpy import ndarray
from typing import Dict, List, Union
from rosbag2numpy.config import params
from rosbag2numpy.config import generalization_model_params as g_params
import wandb
from wandb.keras import WandbCallback
from rosbag2numpy.losses import euclidean_distance_loss,endpoint_loss,costmap_loss_wrapper,loss_wrapper,distance_loss, costmap_loss
from rosbag2numpy.models import conv1x1_endpoint_in_model,conv1x1_endpoint_in_model_costmapout
import time
import os
from rosbag2numpy.train import get_np_test_ds
from tensorflow.keras import backend as K

print(tf.__version__)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


2.3.0
2.3.0


In [2]:
ds_loader = dataset_loader(
    tfrec_dir=params.get("dataset_dir"),
    batch_size=params.get("H_BATCH_SIZE"),
    shuffle_buffer=params.get("H_SHUFFLE_BUFFER"),
    normalize_coords=params.get("normalize_coords"),
    normalize_factor=params.get("normalize_factor")
)
ds_train, ds_valid, ds_test = ds_loader.build_scenario_dataset(consider_scenes=10,no_train_scene=8,no_valid_scene=1,no_test_scene=1)

start = time.time()
#np_ds_test = get_np_test_ds(ds_test=ds_test)
print(f"Test dataset load time: {time.time() - start}")

Scenarios:['/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario1', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario2', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario3', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario4', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario5', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_11', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_12', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_13', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_14', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_15']
Building Dataset.......

Number of records, Train files:610, validation files:109, Test Files:

In [3]:
for X,Y in ds_train.take(1):
    #print(X[-1])
    # costmap, optimized path gt
    print(Y.shape)
    # costmap, grid org res, left bnd, right bnd, car odo, init path, filedetails, gt opt path cost
    print(X[0].shape,X[1].shape,X[2].shape,X[3].shape,X[4].shape,X[5].shape,X[6].shape)


X[0][0][:,:]

(32, 25, 2)
(32, 1536, 1536) (32, 3) (32, 25, 2) (32, 25, 2) (32, 3) (32, 25, 2) (32,)


<tf.Tensor: shape=(1536, 1536), dtype=float16, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)>

In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed


def conc_path_costmap_loss(y_true, y_pred):
    batch_loss = []
    #print(f'shapes: {type(y_true.shape[0]),y_pred.shape}')
    assert y_true.shape[0] == y_pred.shape[0]

    # range of batch size, last batch may not have samples mentioned in batch size
    #for i in range(y_true.shape[0]):
    #    batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 

    with ProcessPoolExecutor() as executor:
        for i in range(y_true.shape[0]):
            param_args = {'y_true':y_true[i][:,:],'y_pred': y_pred[i][:,:]}
            results = {executor.submit(ind_loss,**param_args)}
        for future in as_completed(results):
            batch_loss.append(future.result())

    executor.shutdown(wait=True)
    #print(batch_loss)
    #[batch_loss.append(result) for result in results]
    return tf.convert_to_tensor(batch_loss)


def path_costmap_loss(y_true, y_pred):
    batch_loss = []
    #print(f'shapes: {type(y_true.shape[0]),y_pred.shape}')
    assert y_true.shape[0] == y_pred.shape[0]

    # range of batch size, last batch may not have samples mentioned in batch size
    for i in range(y_true.shape[0]):
        #batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 
        batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 

    #print(batch_loss)
    return tf.convert_to_tensor(batch_loss)

def ind_loss(y_true, y_pred):
    #assuming y true is cost map 1536,1536
    # y_pred is a list of predicted points 25,2

    costmap = y_true
    valid_indices = tf.where(costmap > 0.35) # get indices
    valid_costs = tf.gather_nd(costmap, valid_indices) # get respective values
    valid_costs = tf.cast(valid_costs, dtype=tf.float32) #cast them to float32

    
    allcosts = tf.constant(0.0)
    valid_indices = tf.cast(valid_indices, dtype=tf.float32)
    #print(f"valid_costs:{valid_costs.shape}, valid indices shape : {valid_indices.shape}")

    for i in range(25):

        pred_dist  = K.sqrt(K.sum(K.square(valid_indices - y_pred[i]),axis=1))
        inv_pred_dist = tf.math.reciprocal_no_nan(tf.math.pow(pred_dist,0.1))

        # cost for one point in the path
        cost_for_point = tf.reduce_sum(tf.multiply(inv_pred_dist,valid_costs))
        pred_allcosts = tf.add(allcosts,cost_for_point) if i == 0 else tf.add(pred_allcosts,cost_for_point)
    #print(pred_allcosts.shape)
    
    return K.mean(pred_allcosts)
 

In [5]:
model = conv1x1_endpoint_in_model_costmapout.nn(full_skip= True,params=params)
# Learning rate scheduler

opt = conv1x1_endpoint_in_model_costmapout._get_optimizer(params.get("optimizer"), lr=params.get("lr"))

model.compile(
    optimizer=opt, 
    loss=path_costmap_loss,
    loss_weights=[1]
)

#model.summary()

In [6]:
from tqdm import tqdm
from statistics import mean
# [ip_gridmap,ip_grid_org_res,ip_left_bnd, ip_right_bnd, ip_car_odo, ip_init_path,ip_file_name,ip_gt_cost]

#@tf.function(experimental_relax_shapes=True)
def train_step(data,step):
    #(costmap, grid_org_res, left_bnd, right_bnd, car_odo, init_path, filedetails),(gt_opt_path) = data
    X,Y = data
    if step==1:
        print(X[0].shape,X[1].shape,X[2].shape,X[3].shape,X[4].shape,X[5].shape,X[6].shape)
    with tf.GradientTape() as tape:

        predictions = model(X,training=True)

        cm_loss_pred = path_costmap_loss(X[0],predictions)
        cm_loss_gt = path_costmap_loss(X[0],Y)

        displacement_loss = euclidean_distance_loss(Y,predictions)
        
        path_cost_loss = tf.math.reduce_mean(tf.abs(cm_loss_pred-cm_loss_gt))

        total_loss = 0.2*(path_cost_loss) + displacement_loss
        #print(total_loss)

        grads = tape.gradient(total_loss, model.trainable_weights)

        opt.apply_gradients(zip(grads,model.trainable_weights))
    return total_loss

#@tf.function(experimental_relax_shapes=True)
def train():
    epochs = 30

    for epoch in range(epochs):
        print(f"\n Start of epoch: {epoch+1}")

        #iterate through the batches of dataset.
        for step, (X_batch,Y_batch) in enumerate(tqdm(ds_train)):
            loss_list = []
            loss = train_step((X_batch,Y_batch),step)
            loss_list.append(loss)
        
        print(f"Loss at epoch {epoch+1} is {tf.math.reduce_mean(loss_list)}")

        if (epoch+1)==30:
            model.save('trainmodel.h5')
train()
    

0it [00:00, ?it/s]


 Start of epoch: 1


1it [00:11, 11.91s/it]

(32, 1536, 1536) (32, 3) (32, 25, 2) (32, 25, 2) (32, 3) (32, 25, 2) (32,)


288it [16:11,  3.37s/it]
0it [00:00, ?it/s]

...

1it [00:05,  5.32s/it]

(32, 1536, 1536) (32, 3) (32, 25, 2) (32, 25, 2) (32, 3) (32, 25, 2) (32,)


288it [15:44,  3.28s/it]


Loss at epoch 30 is 4320.31640625


TypeError: save() missing 1 required positional argument: 'filepath'

In [ ]:
history = model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_valid,
    callbacks=[cb_reduce_lr],
)


y_true:Tensor("Cast:0", shape=(None, 1536, 1536), dtype=float32)
y_pred:Tensor("functional_1/reshape_3/Reshape:0", shape=(None, 25, 2), dtype=float32)
cost_gt_opt:Tensor("gt_opt_path_cost:0", shape=(None, 1), dtype=float32)
valid_costs:Tensor("pred_cost/GatherNd:0", shape=(None,), dtype=float32)
y_true:Tensor("IteratorGetNext:9", shape=(None, 25, 2), dtype=float32)
y_pred:Tensor("functional_1/reshape_3/Reshape:0", shape=(None, 25, 2), dtype=float32)
y_true:Tensor("Cast:0", shape=(None, 1536, 1536), dtype=float32)
y_pred:Tensor("functional_1/reshape_3/Reshape:0", shape=(None, 25, 2), dtype=float32)
cost_gt_opt:Tensor("gt_opt_path_cost:0", shape=(None, 1), dtype=float32)
valid_costs:Tensor("pred_cost/GatherNd:0", shape=(None,), dtype=float32)
y_true:Tensor("IteratorGetNext:9", shape=(None, 25, 2), dtype=float32)
y_pred:Tensor("functional_1/reshape_3/Reshape:0", shape=(None, 25, 2), dtype=float32)


_SymbolicException: Inputs to eager execution function cannot be Keras symbolic tensors, but found [<tf.Tensor 'gt_opt_path_cost:0' shape=(None, 1) dtype=float32>]